# 1-Libraries and Dataset downloading

In [1]:
!pip install -q dagshub
!pip install transformers
!pip install langchain_community
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.7/232.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s et

In [21]:
from google.colab import drive
import pandas as pd
import json
from sklearn.preprocessing import MultiLabelBinarizer
import time
import pandas as pd
from google.colab import drive
from transformers import BertTokenizer
PRETRAINED_LM = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
import pickle
from collections import Counter


import re
from langchain import PromptTemplate
from langchain.tools import BaseTool

from langchain.llms import OpenAI
from langchain.chains import LLMChain

import matplotlib.pyplot as plt
import seaborn as sns


In [17]:
# This will prompt for authorization to access your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
file_path_train = '/content/drive/My Drive/Omdena/Sentiment_Analysis/tweets_spanish_train.csv'
file_path_test = '/content/drive/My Drive/Omdena/Sentiment_Analysis/sentiment_beto_sample_bukele.xlsx'
df_train = pd.read_csv(file_path_train)
df_test=pd.read_excel(file_path_test)

In [4]:
len(df_test)

333

In [5]:
df_test.head()

,Post Type,Text,AP,AP + JA Revised
0,Astronaut Post,Dios lo bendiga por ser un gran ser humano,POS,POS
1,Astronaut Post,"Me encanta la humanidad de nuestro astronauta,...",POS,POS
2,Astronaut Post,Dos grandes hombres haciendo historia. Gracias...,POS,POS
3,Astronaut Post,Nayib y Frank dos grandes ejemplo de umildad y...,POS,POS
4,Astronaut Post,A Colombia le falta un precidente con este ...,NEU,POS


In [6]:
# Create a mapping dictionary to streamline LLM classification
sentiment_train_mapping = {
    'positive': 1,
    'neutral': 0,
    'negative': -1
}

sentiment_test_mapping = {
    'POS': 1,
    'NEU': 0,
    'NEG': -1
}

# Apply the mapping to the 'sentiment' column
df_train['sentiment'] = df_train['label'].map(sentiment_train_mapping)
df_test['sentiment_revised'] = df_test['AP + JA Revised'].map(sentiment_test_mapping)
df_test['sentiment'] = df_test['AP'].map(sentiment_test_mapping)

#2-CoT Agentic

In [18]:
class WrapperFrame_v1:
    def __init__(self, client):
        self.client = client

class Sentiment_Agent(WrapperFrame_v1):
    def __init__(self, client):
        super().__init__(client)

        self.sentiment_agent = """

You are an AI language model trained to analyze and classify tweets {tweet} about El Salvador.
Your job is to categorize each tweet into one of three categories based on its sentiment: 1 (positive), 0 (neutral), or -1 (negative) using a
local Perspective from El salvador political scene.

Guidelines:
. 1 (Positive): The tweet expresses a positive sentiment, such as happiness, praise, or admiration.
. 0 (Neutral): The tweet is neutral,. It may present facts, ask questions, or be generally balanced in tone or a statement or just mention a fact or a name.
.-1 (Negative): The tweet expresses a negative sentiment, such as criticism, disappointment, or disapproval.

Examples for Reference:

1 (POS):
. Tweet: "Dios lo bendiga por ser un gran ser humano."
   - Reasoning: The tweet is praising and expressing positive feelings towards someone.

. Tweet: "Me encanta la humanidad de nuestro astronauta, un hombre con gran corazón."
   - Reasoning: The tweet shows love and admiration for the astronaut.

. Tweet: "Dos grandes hombres haciendo historia. Gracias por todo."
   - Reasoning: The tweet appreciates the actions of two men, showing gratitude and positivity.

0 (NEU):
. Tweet: "Tweet Maria Alicia Alas Moreno "
   - Reasoning:The tweet states a fact about the need for a certain type of president without strong positive or negative sentiment.

. Tweet: "Tweet Nuestros obstaculos son mentales"
   - Reasoning: The tweet presents a factual statement about the astronaut's activities without expressing an opinion.

. Tweet: "¿Qué opinan sobre las últimas noticias del presidente?"
   - Reasoning: The tweet is asking a question and does not convey a positive or negative sentiment.

-1 (NEG):
. Tweet: "No estoy de acuerdo con las políticas actuales."
   - Reasoning: The tweet expresses disagreement with current policies, indicating a negative sentiment.

. Tweet: "Es una vergüenza que esto esté sucediendo en nuestro país."
   - Reasoning: The tweet expresses disappointment and criticism about a situation in the country.

. Tweet: "Las decisiones del presidente están dañando la economía."
   - Reasoning: The tweet criticizes the president's decisions, showing a negative sentiment about their impact on the economy.


    Instructions:

      -Read the tweet carefully.

      - Determine the sentiment expressed based on the content, using the following definitions and references:

      - Generate a Json output followin this pattern:

              (
                "classification": 1,0,-1.
                "reasoning: reasoning of the answer in 50 words.
              ).

        """
        self.prompt_sentiment_agent = PromptTemplate(template=self.sentiment_agent, input_variables=["tweet"])

        # Corrected the LLMChain instantiation
        self.llm_chain_sentiment_agent = LLMChain(prompt=self.prompt_sentiment_agent, llm=self.client)

    def run_agent(self,tweet):
        self.tweet = tweet

        try:
            sentiment_output = self.llm_chain_sentiment_agent.run({"tweet": self.tweet})
            return sentiment_output
        except Exception as e:
            print(e)
            return "Error sentiment agent"


In [20]:
class EvaluationProcessPipeline:

    def __init__(self, client, row):
        self.client = client
        self.row = row
        self.text = self.row["Text"]
        self.agent = Sentiment_Agent(self.client)

    def process_tweet(self):
        max_attempts = 1  # Define maximum number of retry attempts
        attempts = 0

        while attempts < max_attempts:
            try:
                # Run the selected Agent
                entry_result = self.agent.run_agent(self.text)
                print("Raw entry_result:", entry_result)  # Debugging line to see what is being returned

                # Attempt to parse the JSON data
                data = json.loads(entry_result)
                labels = data["classification"]
                print("Extracted labels:", labels)  # Debugging line to confirm label extraction
                return labels  # Successful parsing and return, exit loop
            except (json.JSONDecodeError, Exception) as error:
                # Handle both JSON parsing errors and other exceptions
                print(f"Error encountered (attempt {attempts + 1}):", error)
                attempts += 1
                time.sleep(1)  # Sleep before retrying

        print("Failed after", max_attempts, "attempts.")
        return []



In [ ]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

In [11]:
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(




#2.1- Inference Pipeline


In [14]:
from sklearn.preprocessing import MultiLabelBinarizer
import time

In [ ]:
predicted_values=[]
real_values_revised=[]
for i,entry in enumerate (df_test.loc[:,'Text']):
    print(i)
    agent = EvaluationProcessPipeline(llm, df_test.iloc[i])
    print('Tweet',df_test.loc[i,'Text'])
    label = agent.process_tweet()
    predicted_values.append(label)
    real_values_revised.append(df_test.iloc[i]['sentiment_revised'])
    print('Real Value Revisited',df_test.iloc[i]['sentiment_revised'])
    time.sleep(1)


